In [1]:
# importing libraries

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-10-16 06:33:07.726676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 06:33:07.855826: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 06:33:07.858273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 06:33:09.707667: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Directory containing your MRI images with 'tumor' and 'no_tumor' subdirectories
data_dir = "./data/train"


In [3]:
# Function to load and preprocess your MRI dataset
def load_and_preprocess_data(data_dir, target_size=(150, 150), batch_size=32):
    # Create an ImageDataGenerator for data augmentation and preprocessing
    datagen = ImageDataGenerator(
        rescale=1.0 / 255.0,  # Normalize pixel values
        rotation_range=15,  # Augmentation: Randomly rotate images
        width_shift_range=0.1,  # Augmentation: Randomly shift width
        height_shift_range=0.1,  # Augmentation: Randomly shift height
        shear_range=0.2,  # Augmentation: Randomly shear images
        zoom_range=0.2,  # Augmentation: Randomly zoom images
        horizontal_flip=True,  # Augmentation: Randomly flip images horizontally
    )

    # Load and preprocess the training data
    train_data_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,  # Resize images to a common size
        batch_size=batch_size,
        class_mode="binary",  # Assuming two classes: 'tumor' and 'no_tumor'
        subset="training",  # You can further split this into training and validation
    )

    return train_data_generator


In [4]:
# Load and preprocess the MRI data
train_data_generator = load_and_preprocess_data(data_dir)

# Define the CNN model
model = keras.Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(1, activation="sigmoid"),
    ]
)

Found 253 images belonging to 2 classes.


In [5]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [6]:
# Train the model
model.fit(
    train_data_generator, epochs=10, validation_data=None
)  # Pass validation_data for validation


Epoch 1/10


2023-10-16 06:34:05.567011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-10-16 06:34:08.437367: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89718784 exceeds 10% of free system memory.
2023-10-16 06:34:08.555115: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42467328 exceeds 10% of free system memory.
2023-10-16 06:34:09.218889: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42467328 exceeds 10% of free system memory.
2023-10-16 06:34:09.676077: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2023-10-16 06:34:09.677113: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89718784 exceeds 10% of

8/8 [==============================] - 14s 1s/step - loss: 0.7232 - accuracy: 0.5731
Epoch 2/10
8/8 [==============================] - 11s 1s/step - loss: 0.6315 - accuracy: 0.6443
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6122 - accuracy: 0.6759
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 0.5736 - accuracy: 0.7233
Epoch 5/10
8/8 [==============================] - 10s 1s/step - loss: 0.5508 - accuracy: 0.7668
Epoch 6/10
8/8 [==============================] - 10s 1s/step - loss: 0.5328 - accuracy: 0.7312
Epoch 7/10
8/8 [==============================] - 10s 1s/step - loss: 0.5277 - accuracy: 0.7391
Epoch 8/10
8/8 [==============================] - 10s 1s/step - loss: 0.5225 - accuracy: 0.7668
Epoch 9/10
8/8 [==============================] - 9s 1s/step - loss: 0.4855 - accuracy: 0.7708
Epoch 10/10
8/8 [==============================] - 9s 1s/step - loss: 0.5260 - accuracy: 0.7708


In [7]:
# Save the model
model.save("model.h5")
